# 02: Investigating Particular Buildings | RTEM Hackathon
There are particular buildings that have an interesting array of equipment sensors. Let's single out particular
buildings to see if we get any ideas!

## Import Packages
Let's start by importing the stuff we need.
This includes packages to work with JSON data, as well as the Onboard Client

In [10]:
# Import the packages we need to do our analysis
import json

import pandas as pd
from onboard.client import RtemClient

## Be Safe With Your Creds
Just because this is a Hackathon and a personal project doesn't mean we get to flippantly broadcast our API Keys to
the world. I want to publish this code to GitHub so let's be responsible and import our keys using configparser.

In [11]:
# Load API Key using configparser
import configparser

config = configparser.ConfigParser()
config.read('../config.ini')
api_key = config['DEFAULT']['API_KEY']

## Use Cached Data?
Some of these client requests take a long time to return data, so in the interests of time I saved local versions by
writing DataFrames to JSON files.

In [12]:
# If use_local set to true, try to find the local file and use that.
use_local = True

## Initialise the Onboard Client
Then check to see if it's working properly.

In [13]:
# Initialise Onboard Client
client = RtemClient(api_key=api_key)

# Check that everything is in order
whoami = client.whoami()
print(f"""
    Result: {whoami['result'].upper()}
    Scope: {whoami['apiKeyScopes']}
    Version: {whoami['apiVersion']}
    User: {whoami['userInfo']['full_name']}
    Org: {whoami['userInfo']['org_short_name']}
""")


    Result: OK
    Scope: ['buildings:read', 'general', 'auth']
    Version: 2022-04-14
    User: Daniel Lawson
    Org: RTEM Submission



## Which buildings are interesting enough to look further into?
- Building 110: Selfhelp K1? Cafeteria?
- Building 118: Data points for Assorted Plug Loads
- Building 120: 1700 Broadway, 42-story office tower, Con Ed Steam Meter
- Building 127: Selfhelp K2?
- Building 128: 800 Grand Concourse
- Building 134: Has Appliances, Telephones, Electric Chairs?, Mix Lab, Cash register etc...
- Building 141: Has cost and usage data for plant equipment
- Building 230: Seems to be a Fitness Centre /or building with a fitness centre? Elliptical, Treadmill power meters
- Building 423: Seems to have water meters for the baths and kitchen sinks?
- Building 426: 480 W 42nd St Signature Theatre
- Building 438: AOL Time Warner Building
- Building 440: Floor Level Power Usage
- Building 441: 225/235/245/255 E 149th St, Temperature Zone Sensors
- Building 445: 450 W 147th - Boiler Target/Effective
- Building 477: Procedure & Animal Rooms?
- Building 484: Broadway? Commercial Office
- Building 487: Grand Central? Commercial Office
- Building 503: Madison - Floor Temperature Measurements
- Building 472: SUNY Geneseo - Zone Temp HVAC
- Building 256: Manufacturing, Import/Export
- Building 420: Memorial Sloan Kettering Cancer Center

### Concept: Con Ed Steam System?
Could you quantify whether this system is better or worse than other heating systems?

## Building 441: Multifunctional Mixed-Use Building
225, 235, 245, 255 E 149th St New York

In [14]:
if use_local:
    with open('../api/buildings/441.json', 'r') as f:
        j = json.load(f)
        b_441 = pd.json_normalize(j)
else:
    b_441 = client.get_building_equipment(441)
b_441

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,28797,441,boiler-1,1,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310029, 'building_id': 441, 'last_upda...","[boiler, hvac]"
1,28798,441,boiler-2,2,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310079, 'building_id': 441, 'last_upda...","[boiler, hvac]"
2,28799,441,boiler-3,3,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310108, 'building_id': 441, 'last_upda...","[boiler, hvac]"
3,28800,441,boiler-4,4,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310144, 'building_id': 441, 'last_upda...","[boiler, hvac]"
4,28801,441,boiler-5,5,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310152, 'building_id': 441, 'last_upda...","[boiler, hvac]"
5,28802,441,domesticWaterPump-site,site,Pump,32,PUMP,pump,Domestic Water Pump,22.0,domesticWaterPump,1,None,None,None,None,[28803],[],"[{'id': 310164, 'building_id': 441, 'last_upda...","[pump, hvac, domesticWaterPump]"
6,28803,441,hotWaterPlant,None,Hot Water Plant,30,HWS,hotWaterPlant,None,NaN,None,1,None,None,None,None,[],"[28797, 28798, 28799, 28800, 28801, 28802, 288...","[{'id': 310210, 'building_id': 441, 'last_upda...","[hvac, hotWaterPlant]"
7,28804,441,hotWaterPump-1,1,Pump,32,PUMP,pump,Hot Water Pump,25.0,hotWaterPump,1,None,None,None,None,[28803],[],"[{'id': 310232, 'building_id': 441, 'last_upda...","[pump, hvac, hotWaterPump]"
8,28805,441,hotWaterPump-2,2,Pump,32,PUMP,pump,Hot Water Pump,25.0,hotWaterPump,1,None,None,None,None,[28803],[],"[{'id': 310238, 'building_id': 441, 'last_upda...","[pump, hvac, hotWaterPump]"
9,28806,441,hotWaterPump-4,4,Pump,32,PUMP,pump,Hot Water Pump,25.0,hotWaterPump,1,None,None,None,None,[28803],[],"[{'id': 310250, 'building_id': 441, 'last_upda...","[pump, hvac, hotWaterPump]"


In [15]:
# Get all points for building 441
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_441_points = []
        for line in j:
            if line['building_id'] == 441:
                b_441_points.append(line)

In [16]:
df_b_441_points = pd.DataFrame(b_441_points).set_index('id')
df_b_441_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
310029,441,1.560571e+12,1.550542e+12,None,None,None,245 E 149th St. Boiler-1 Fault Code,None,?,?,23,0,Virtual,434,18,b115b5053493748f7076ad09f69c4456,nyserda+0000088737+boiler-1+245 E 149th St. Bo...,None,28797
310009,441,1.560485e+12,1.550701e+12,None,None,None,225 E 149th St. Boiler-1 Exhaust Temp,None,F,f,2,183.55,Exhaust Air Temperature,86,1,4a1a11502acee3078352da312c40c262,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,None,28797
310010,441,1.560571e+12,1.550504e+12,None,None,None,225 E 149th St. Boiler-1 Fault Code,None,?,?,23,0,Virtual,434,18,55882633fe8fb5aac7dac643c99b1af5,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,None,28797
310011,441,1.560485e+12,1.550504e+12,None,None,None,225 E 149th St. Boiler-1 FFWD Temp,None,?,?,23,46.90,Virtual,434,18,831a8df09b848f53e807fa0204c45eaa,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,None,28797
310012,441,1.560485e+12,1.550504e+12,None,None,None,225 E 149th St. Boiler-1 Fire Rate In,None,?,?,23,91,Virtual,434,18,aa147c829c7ed55b6e93cdbbb8b43d55,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,None,28797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311251,441,1.609476e+12,1.561954e+12,None,None,None,225 E 149th St. Sensor Status HHW Return Tempe...,None,?,?,23,1,Virtual,434,18,6c03f4d6c18f152855886c8f4ac0c40f,nyserda+0000088737+virtual+225 E 149th St. Sen...,None,28814
344965,441,1.560486e+12,1.554729e+12,None,None,None,225 E 149th St. Water Meter Water Meter,None,gal,gal,68,11178,Water Consumption,224,22,5f290fd367657b5b3a79c4eed09150c7,nyserda+0000088737+waterMeterLoad-site+225 E 1...,None,30572
344964,441,1.609476e+12,1.561954e+12,None,None,None,225 E 149th St. Basement Common Meter Water Ga...,None,gal,gal,68,0,Water Consumption,224,22,e03c320eb45ad0fed2bf074ee657aef5,nyserda+0000088737+waterMeterLoad-site+225 E 1...,None,30572


## Building 110: 45-25 Kissena Blvd Queens NY
- Selfhelp Aged Care Home K1
https://www.propertyshark.com/mason/Property/471854/45-25-Kissena-Blvd-Queens-NY-11355/

In [17]:
if use_local:
    with open('../api/buildings/110.json', 'r') as f:
        j = json.load(f)
        b_110 = pd.json_normalize(j)
else:
    b_110 = client.get_building_equipment(110)
b_110

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,31776,110,1,meter-water,Meter,31,METER,meter,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 358168, 'building_id': 110, 'last_upda...",[meter]
1,17360,110,Site-Sensor,Site-Sensor,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 284172, 'building_id': 110, 'last_upda...",[site]
2,17361,110,Site-Virtual,Site-Virtual,Virtual,46,VIRT,virtual,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 284193, 'building_id': 110, 'last_upda...",[]
3,17362,110,meter-electric,meter-electric,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 284225, 'building_id': 110, 'last_upda...","[meter, elecMeter]"
4,27027,110,ahu-1,1,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[],"[{'id': 284194, 'building_id': 110, 'last_upda...","[ahu, hvac]"
5,27028,110,ahu-2,2,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[],"[{'id': 284195, 'building_id': 110, 'last_upda...","[ahu, hvac]"
6,27029,110,boiler-1,1,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 284196, 'building_id': 110, 'last_upda...","[boiler, hvac]"
7,27030,110,boiler-2,2,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 284197, 'building_id': 110, 'last_upda...","[boiler, hvac]"
8,27031,110,chilledWaterPlant,None,Chilled Water Plant,20,CHWS,chilledWaterPlant,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 284202, 'building_id': 110, 'last_upda...","[hvac, chilledWaterPlant]"
9,27032,110,hotWaterPlant,None,Hot Water Plant,30,HWS,hotWaterPlant,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 284222, 'building_id': 110, 'last_upda...","[hvac, hotWaterPlant]"


In [18]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_110_points = []
        for line in j:
            if line['building_id'] == 110:
                b_110_points.append(line)

In [19]:
df_b_110_points = pd.DataFrame(b_110_points).set_index('id')
df_b_110_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
358168,110,1.609476e+12,1.601525e+12,None,None,None,Selfhelp K1 Meter-Water Gallons Total,None,gallons,None,68,0,Water Consumption,224,22,92669d9df580035389cc1426d66dadbf,nyserda+0000089088+meter-water+Selfhelp K1 Met...,None,31776
358167,110,1.609476e+12,1.601525e+12,None,None,None,Selfhelp K1 Meter-Water Makeup Gallons Total,None,gallons,None,68,0,Water Consumption,224,22,2aec8bb3eeecaa159bf3e3d97126fa4a,nyserda+0000089088+meter-makeUpWater+Selfhelp ...,None,31776
358166,110,1.609476e+12,1.601525e+12,None,None,None,Selfhelp K1 Meter-Gas Therms Total,None,Therms,None,70,10.35,Gas Consumption,400,14,b6577b83bd54db529b2bd82fac182e8f,nyserda+0000089088+meter-gas+Selfhelp K1 Meter...,None,31776
284172,110,1.604202e+12,1.601525e+12,None,None,None,Selfhelp K1 Zone Space Temperature.9,None,F,f,2,65.78,Zone Temperature,77,1,4358ef77c929b2a72849ac7da49eac27,nyserda+0000089088+Site-Sensor+Selfhelp K1 Zon...,None,17360
284171,110,1.604202e+12,1.601525e+12,None,None,None,Selfhelp K1 Zone Space Temperature.8,None,F,f,2,76.70,Zone Temperature,77,1,ee728b6c5f81eb9dae4fad9570b16654,nyserda+0000089088+Site-Sensor+Selfhelp K1 Zon...,None,17360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284264,110,1.609474e+12,1.593576e+12,None,None,None,Manual Boost Time.2,None,?,?,23,10,Virtual,434,18,bc2f8562fa8bbe3a9cf69992f973f945,nyserda+0000089088+site-Virtual+Manual Boost T...,None,27036
358169,110,1.601524e+12,1.593576e+12,None,None,None,Water Meter Room Meter-Water Gallons Total,None,gallons,None,68,0,Water Consumption,224,22,42b181fd3c3861c02dfb9f418629eb96,nyserda+0000089088+meter-water+Water Meter Roo...,None,17365
194175,110,1.541076e+12,1.535774e+12,,None,,,Selfhelp K1 Meters Water Meter Dmarc,ft3,None,27,4.48,Water Consumption,224,22,fdaaaaca6af8392d96517c8537f0bb36,nyserda+0000089088+meter-water+Selfhelp K1 Met...,None,17365


## Building 127: K2? Possible Kissena Blvd??
- Selfhelp Kissena Aged Care Homes

In [20]:
if use_local:
    with open('../api/buildings/127.json', 'r') as f:
        j = json.load(f)
        b_127 = pd.json_normalize(j)
else:
    b_127 = client.get_building_equipment(127)
b_127

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,17863,127,Site-Virtual,Site-Virtual,Virtual,46,VIRT,virtual,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 284527, 'building_id': 127, 'last_upda...",[]
1,17864,127,boiler1,boiler1,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[17865],[],"[{'id': 197306, 'building_id': 127, 'last_upda...","[boiler, hvac]"
2,17865,127,hotWaterPlant,None,Hot Water Plant,30,HWS,hotWaterPlant,None,NaN,None,1,None,None,None,None,[],"[17866, 17864]","[{'id': 284313, 'building_id': 127, 'last_upda...","[hvac, hotWaterPlant]"
3,17866,127,boiler2,boiler2,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[17865],[],"[{'id': 197308, 'building_id': 127, 'last_upda...","[boiler, hvac]"
4,17867,127,meter-electric,meter-electric,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 284319, 'building_id': 127, 'last_upda...","[meter, elecMeter]"
5,17868,127,meter-gas,meter-gas,Meter,31,METER,meter,Gas Meter,17.0,gasMeter,5,None,None,None,gasMeter,[],[],"[{'id': 197311, 'building_id': 127, 'last_upda...","[meter, gasMeter]"
6,17869,127,meter-makeUpWater,meter-makeUpWater,Meter,31,METER,meter,Water Meter,19.0,waterMeter,5,None,None,None,makeUpWaterMeter,[],[],"[{'id': 284320, 'building_id': 127, 'last_upda...","[meter, waterMeter]"
7,17870,127,meter-water,meter-water,Meter,31,METER,meter,Water Meter,19.0,waterMeter,5,None,None,None,waterMeter,[],[],"[{'id': 197313, 'building_id': 127, 'last_upda...","[meter, waterMeter]"
8,27037,127,pump-1,1,Pump,32,PUMP,pump,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 284321, 'building_id': 127, 'last_upda...","[pump, hvac]"
9,27038,127,pump-2,2,Pump,32,PUMP,pump,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 284322, 'building_id': 127, 'last_upda...","[pump, hvac]"


In [21]:
# Get all points for building 127
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_127_points = []
        for line in j:
            if line['building_id'] == 127:
                b_127_points.append(line)

df_b_127_points = pd.DataFrame(b_127_points).set_index('id')
df_b_127_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
284527,127,1.601524e+12,1.593576e+12,None,None,None,Water Meter Room Meter - Water Makeup Make Up ...,None,?,?,23,0,Virtual,434,18,fcfc69b309b0666573cece9c15c610b5,nyserda+0000089089+Site-Virtual+Water Meter Ro...,None,17863
284526,127,1.609476e+12,1.601525e+12,None,None,None,Selfhelp K2 Controller Heat Timer System Run-on,None,?,?,23,0,Virtual,434,18,ed6f11bd71cb044125b124d8fa45c618,nyserda+0000089089+Site-Virtual+Selfhelp K2 Co...,None,17863
284525,127,1.609476e+12,1.593576e+12,None,None,None,Selfhelp K2 Controller Heat Timer System Relay,None,?,?,23,AAAB,Virtual,434,18,7b99411a3dc9f54c9cfe22b79aaf85c3,nyserda+0000089089+Site-Virtual+Selfhelp K2 Co...,None,17863
284524,127,1.609476e+12,1.601525e+12,None,None,None,Selfhelp K2 Controller Heat Timer Standby Delay,None,?,?,23,10,Virtual,434,18,3dc339619ab1e8c4a5b9f0068879a4ea,nyserda+0000089089+Site-Virtual+Selfhelp K2 Co...,None,17863
284523,127,1.609476e+12,1.601525e+12,None,None,None,Selfhelp K2 Controller Heat Timer Setback,None,?,?,23,0,Virtual,434,18,8163a2f447cbfb71fb166fc789a08db2,nyserda+0000089089+Site-Virtual+Selfhelp K2 Co...,None,17863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284411,127,1.609476e+12,1.604203e+12,None,None,None,Selfhelp K2 Apt-2Q Zone Space Temperature,None,F,f,2,70.16,Zone Temperature,77,1,e1d036efa8f4314215bc1ad9327a02fe,nyserda+0000089089+Site-Sensor+Selfhelp K2 Apt...,None,17862
284410,127,1.609476e+12,1.604203e+12,None,None,None,Selfhelp K2 Apt-2P Zone Space Temperature,None,F,f,2,70.45,Zone Temperature,77,1,d0d83c4e6cbed0225350ecc5d1ebb40a,nyserda+0000089089+Site-Sensor+Selfhelp K2 Apt...,None,17862
284372,127,1.609476e+12,1.604203e+12,None,None,None,Selfhelp K2 Apt-11D Zone Space Temperature,None,F,f,2,74.75,Zone Temperature,77,1,8cb444d4b773e71987f93a7ad64485db,nyserda+0000089089+Site-Sensor+Selfhelp K2 Apt...,None,17862


## Building 120:

In [22]:
if use_local:
    with open('../api/buildings/120.json', 'r') as f:
        j = json.load(f)
        b_120 = pd.json_normalize(j)
else:
    b_120 = client.get_building_equipment(120)
b_120

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,17669,120,Site-Sensor,Site-Sensor,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,,,[],[],"[{'id': 196685, 'building_id': 120, 'last_upda...",[site]
1,17670,120,Site-Virtual,Site-Virtual,Virtual,46,VIRT,virtual,None,NaN,None,5,None,None,,,[],[],"[{'id': 196686, 'building_id': 120, 'last_upda...",[]
2,17671,120,ac1,ac1,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,Base Bldg AC-1,[],[],"[{'id': 196698, 'building_id': 120, 'last_upda...","[ahu, hvac]"
3,17672,120,ac2,ac2,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,Base Bldg AC-2,[],[],"[{'id': 196722, 'building_id': 120, 'last_upda...","[ahu, hvac]"
4,17673,120,ac3,ac3,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,Base Bldg AC-3,[],[],"[{'id': 196729, 'building_id': 120, 'last_upda...","[ahu, hvac]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,17784,120,heatingValve1,heatingValve1,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,,Base Bldg HV-1,[],[],"[{'id': 197078, 'building_id': 120, 'last_upda...",[site]
116,17785,120,heatingValve2,heatingValve2,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,,Base Bldg HV-2,[],[],"[{'id': 197081, 'building_id': 120, 'last_upda...",[site]
117,17786,120,meter1-steam,meter1-steam,Meter,31,METER,meter,Steam Meter,18.0,steamMeter,5,None,None,,Base Bldg Steam Meters Con Ed Steam Meter 1,[],[],"[{'id': 197084, 'building_id': 120, 'last_upda...","[meter, steamMeter]"
118,17787,120,meter2-steam,meter2-steam,Meter,31,METER,meter,Steam Meter,18.0,steamMeter,5,None,None,,Base Bldg Steam Meters Con Ed Steam Meter 2,[],[],"[{'id': 197085, 'building_id': 120, 'last_upda...","[meter, steamMeter]"


# Get all points for building 120

In [23]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_120_points = []
        for line in j:
            if line['building_id'] == 120:
                b_120_points.append(line)

df_b_120_points = pd.DataFrame(b_120_points).set_index('id')
df_b_120_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
196682,120,1.543640e+12,1.535774e+12,,None,,,1700 Broadway - Base Bldg Misc 12th Floor Elev...,F,None,2,81.96,Zone Temperature,77,1,1c9291c647ac9381813a03fa575ac694,nyserda+0000075506+Site-Sensor+1700 Broadway -...,None,17669
196681,120,1.543640e+12,1.535774e+12,,None,,,1700 Broadway - Base Bldg Lower House Discharg...,F,None,2,80,Discharge Air Temperature,79,1,9e0c707de668ac39a5e285608615c84b,nyserda+0000075506+Site-Sensor+1700 Broadway -...,None,17669
196683,120,1.543640e+12,1.535774e+12,,None,,,1700 Broadway - Base Bldg Misc 27th Floor Elev...,F,None,2,77.2,Zone Temperature,77,1,1f7f733b3d5123ba6a77dfb47637b26c,nyserda+0000075506+Site-Sensor+1700 Broadway -...,None,17669
196678,120,1.543640e+12,1.535774e+12,,None,,,1700 Broadway - Base Bldg Global North Outside...,F,None,2,327.68,Outside Air Temperature,76,1,9e45a4bdc7b7b589c76c8f83e835c838,nyserda+0000075506+Site-Sensor+1700 Broadway -...,None,17669
196680,120,1.543640e+12,1.535774e+12,,None,,,1700 Broadway - Base Bldg Global South Outside...,F,None,2,40.84,Outside Air Temperature,76,1,367c30f48cf70193697f3b3ee34030c6,nyserda+0000075506+Site-Sensor+1700 Broadway -...,None,17669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197081,120,1.543640e+12,1.535774e+12,,None,,,1700 Broadway - Base Bldg HV-2 Occupied Status,Occ,None,18,false,Occupancy Status,177,10,f0e4b19160f28379c8ed3f1557059520,nyserda+0000075506+heatingValve2+1700 Broadway...,None,17785
197083,120,1.543640e+12,1.535774e+12,,None,,,1700 Broadway - Base Bldg HV-2 Supply Fan Status,On/Off,None,20,true,Status,121,11,a5e323f0f834b3c7f337398ad993b4b3,nyserda+0000075506+heatingValve2+1700 Broadway...,None,17785
197084,120,1.543640e+12,1.535774e+12,,None,,,1700 Broadway - Base Bldg Steam Meters Con Ed ...,kWh,None,16,0,Electric Consumption,178,14,29e72569443f0dc0561b7a8c41b00c6d,nyserda+0000075506+meter1-steam+1700 Broadway ...,None,17786


## Building 477: Animal & Procedure Rooms

In [24]:
if use_local:
    with open('../api/buildings/477.json', 'r') as f:
        j = json.load(f)
        b_477 = pd.json_normalize(j)
else:
    b_477 = client.get_building_equipment(477)
b_477

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,31777,477,ahu-1,1,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[31799],"[{'id': 358177, 'building_id': 477, 'last_upda...","[ahu, hvac]"
1,31778,477,ahu-2,2,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[31800],"[{'id': 358190, 'building_id': 477, 'last_upda...","[ahu, hvac]"
2,31779,477,ahu-5,5,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[],"[{'id': 358193, 'building_id': 477, 'last_upda...","[ahu, hvac]"
3,31780,477,ahu-6,6,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[],"[{'id': 358196, 'building_id': 477, 'last_upda...","[ahu, hvac]"
4,31781,477,ahu-ACS1,ACS1,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],"[32152, 32153]","[{'id': 358230, 'building_id': 477, 'last_upda...","[ahu, hvac]"
5,31782,477,ahu-ACS11,ACS11,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[31802],"[{'id': 358198, 'building_id': 477, 'last_upda...","[ahu, hvac]"
6,31783,477,ahu-ACS2,ACS2,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],"[31803, 31804]","[{'id': 358249, 'building_id': 477, 'last_upda...","[ahu, hvac]"
7,31786,477,ahu-ACS3,ACS3,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[31805],"[{'id': 358269, 'building_id': 477, 'last_upda...","[ahu, hvac]"
8,31789,477,ahu-ACS4,ACS4,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],"[31807, 31808]","[{'id': 358289, 'building_id': 477, 'last_upda...","[ahu, hvac]"
9,31792,477,chilledWaterPlant-Primary,Primary,Chilled Water Plant,20,CHWS,chilledWaterPlant,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 358296, 'building_id': 477, 'last_upda...","[hvac, chilledWaterPlant]"


In [25]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_477_points = []
        for line in j:
            if line['building_id'] == 477:
                b_477_points.append(line)

df_b_477_points = pd.DataFrame(b_477_points).set_index('id')
df_b_477_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
358177,477,1.614574e+12,1.609477e+12,None,None,None,ES.AHU-01_Outdoor Air Damper Minimum Position_...,None,None,%,3,4.8,Min Outside Air Damper Position,571,7,910f4b8c9db9b4b6c05d8d02bfb7f159,nyserda+0000159950+ahu-1+ES.AHU-01_Outdoor Air...,None,31777
358179,477,1.614574e+12,1.609477e+12,None,None,None,ES.AHU-01_Supply Air Temperature_ELM.RTU101:DI...,None,None,f,2,75.5,Supply Air Temperature,425,1,0f1b51e816ae55a81b76f1b6ce7eac6d,nyserda+0000159950+ahu-1+ES.AHU-01_Supply Air ...,None,31777
358178,477,1.614574e+12,1.609477e+12,None,None,None,ES.AHU-01_Room Temperature Setpoint_ELM.RTU101...,None,None,f,2,70.0,Zone Temperature Setpoint,122,1,e65371d568f32b0ca736592082ea3b5f,nyserda+0000159950+ahu-1+ES.AHU-01_Room Temper...,None,31777
358176,477,1.614574e+12,1.609477e+12,None,None,None,ES.AHU-01_Occupied Mode_ELM.RTU101:DAY.NGT,None,None,occ,18,0.0,Occupancy Command,286,10,73142b5c02de894da1855f89a1c95a4e,nyserda+0000159950+ahu-1+ES.AHU-01_Occupied Mo...,None,31777
358175,477,1.611298e+12,1.609477e+12,None,None,None,ES.AHU-01_Mixing Box Dampers Command_ELM.RTU10...,None,None,%,3,14.0,Mixed Air Damper Position,149,7,61d56e4e4ae038857bf73e8eb876d43f,nyserda+0000159950+ahu-1+ES.AHU-01_Mixing Box ...,None,31777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358490,477,1.622519e+12,1.594786e+12,None,None,None,ACS 04B Supply Fan Speed Command_A04BSF,None,None,%,3,64.09,Speed,158,17,a23652053099ddbf36df7d5487660500,nyserda+0000159950+supplyFan-ACS4B+ACS 04B Sup...,None,31808
359789,477,1.622519e+12,1.595214e+12,None,None,None,ACS 01A Supply Fan Start/Stop Command_A01ASS,None,None,on/off,20,1.0,Command,166,11,5a9aaf6dafd26a96ef66d14b6fd267e5,nyserda+0000159950+supplyFan-ACS1A+ACS 01A Sup...,None,32152
359788,477,1.622519e+12,1.595214e+12,None,None,None,ACS 01A Supply Fan Speed Command_A01ASF,None,None,%,3,67.91,Speed,158,17,e5ce666d16a1ee5bca89f57b567ec510,nyserda+0000159950+supplyFan-ACS1A+ACS 01A Sup...,None,32152


# Building 141: Cost & Usage Data for Plant Equipment

In [26]:
if use_local:
    with open('../api/buildings/141.json', 'r') as f:
        j = json.load(f)
        b_141 = pd.json_normalize(j)
else:
    b_141 = client.get_building_equipment(141)
b_141

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,18105,141,chilledWaterPump-lobby,chilledWaterPump-lobby,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,,Lobby Chiller/Pump,[],[],"[{'id': 198473, 'building_id': 141, 'last_upda...","[pump, hvac, chilledWaterPump]"
1,18106,141,chilledWaterPump36-2,chilledWaterPump36-2,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,,Pump 36-2 (CHW),[],[],"[{'id': 198476, 'building_id': 141, 'last_upda...","[pump, hvac, chilledWaterPump]"
2,18107,141,chilledWaterPump5,chilledWaterPump5,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,,Pump 5 (Chilled Water),[],[],"[{'id': 198478, 'building_id': 141, 'last_upda...","[pump, hvac, chilledWaterPump]"
3,18108,141,chilledWaterPump6,chilledWaterPump6,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,,Pump 6 (Chilled Water),[],[],"[{'id': 198480, 'building_id': 141, 'last_upda...","[pump, hvac, chilledWaterPump]"
4,18109,141,chiller3,chiller3,Chiller,21,CH,chiller,None,NaN,None,1,None,None,,Chiller 3,[],[],"[{'id': 198482, 'building_id': 141, 'last_upda...","[chiller, hvac]"
5,18110,141,condenserWaterPump-secondary,condenserWaterPump-secondary,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,,Secondary Condenser Water Pumps 28/29,[],[],"[{'id': 198484, 'building_id': 141, 'last_upda...","[pump, hvac, condenserWaterPump]"
6,18111,141,condenserWaterPump36-1,condenserWaterPump36-1,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,,Pump 36-1 (CDW),[],[],"[{'id': 198486, 'building_id': 141, 'last_upda...","[pump, hvac, condenserWaterPump]"
7,18112,141,condenserWaterPump8,condenserWaterPump8,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,,Pump 8 (Condenser Water),[],[],"[{'id': 198488, 'building_id': 141, 'last_upda...","[pump, hvac, condenserWaterPump]"
8,18113,141,condenserWaterPump9,condenserWaterPump9,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,,Pump 9 (Condenser Water),[],[],"[{'id': 198489, 'building_id': 141, 'last_upda...","[pump, hvac, condenserWaterPump]"
9,18114,141,elecPanel-lvp,elecPanel-lvp,Electrical Panel,24,ELEC,panel,None,NaN,None,1,None,None,,UP-12UC & 12UD LVP-12-(1-3),[],[],"[{'id': 198492, 'building_id': 141, 'last_upda...",[panel]


In [27]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_141_points = []
        for line in j:
            if line['building_id'] == 141:
                b_141_points.append(line)

df_b_141_points = pd.DataFrame(b_141_points).set_index('id')
df_b_141_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
198473,141,1.552020e+12,1.520399e+12,,None,,,Lobby Chiller/Pump Total Cost ($),,None,23,0.13,Virtual,434,18,69ab5fbf6c0ac735343ef796ea71e05b,nyserda+0000084481+chilledWaterPump-lobby+Lobb...,None,18105
198474,141,1.552020e+12,1.520399e+12,,None,,,Lobby Chiller/Pump Active Power(kW),kW,None,17,2.94,Real Power,221,13,d97fdf24df7e6d0977888cd0ecbc7ecc,nyserda+0000084481+chilledWaterPump-lobby+Lobb...,None,18105
198476,141,1.552020e+12,1.520399e+12,,None,,,Pump 36-2 (CHW) Active Power(kW),kW,None,17,20.85,Real Power,221,13,88b7faa51880632556e75b9983829ba5,nyserda+0000084481+chilledWaterPump36-2+Pump 3...,None,18106
198475,141,1.552020e+12,1.520399e+12,,None,,,Pump 36-2 (CHW) Total Cost ($),,None,23,0.93,Virtual,434,18,2aeb8562ee63924cda408e015904f008,nyserda+0000084481+chilledWaterPump36-2+Pump 3...,None,18106
198478,141,1.552020e+12,1.520399e+12,,None,,,Pump 5 (Chilled Water) Active Power(kW),kW,None,17,0.0,Real Power,221,13,6e0892ba6d2c2b6b8caab5ad2a59b667,nyserda+0000084481+chilledWaterPump5+Pump 5 (C...,None,18107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198550,141,1.552020e+12,1.520399e+12,,None,,,TP 1-4 (Tenant Pumps) Active Power(kW),kW,None,17,13.78,Real Power,221,13,8f67e164ff408d5086e698edb925302f,nyserda+0000084481+pump-tenant+TP 1-4 (Tenant ...,None,18143
198552,141,1.552020e+12,1.520399e+12,,None,,,Pump 7 (Standby Pump) Active Power(kW),kW,None,17,0.0,Real Power,221,13,51be8b2b97a6eeacd31cf7607cddd682,nyserda+0000084481+pump7-standby+Pump 7 (Stand...,None,18144
198551,141,1.552020e+12,1.520399e+12,,None,,,Pump 7 (Standby Pump) Total Cost ($),,None,23,0.0,Virtual,434,18,7689a39fe518e88cd8d5e17ad48ae6ae,nyserda+0000084481+pump7-standby+Pump 7 (Stand...,None,18144


# Building 230: Fitness Centre?
Seems to be a Fitness Centre /or building with a fitness centre? Elliptical, Treadmill power meters

In [28]:
if use_local:
    with open('../api/buildings/230.json', 'r') as f:
        j = json.load(f)
        b_230 = pd.json_normalize(j)
else:
    b_230 = client.get_building_equipment(230)
b_230

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,19355,230,Site-Virtual,Site-Virtual,Virtual,46,VIRT,virtual,None,NaN,None,5,None,None,,,[],[],"[{'id': 206084, 'building_id': 230, 'last_upda...",[]
1,19356,230,meter-ac-hallway,meter-ac-hallway,Meter,31,METER,meter,None,NaN,None,5,None,None,,,[],[],"[{'id': 206164, 'building_id': 230, 'last_upda...",[meter]
2,19357,230,meter-ac-mechRoom,meter-ac-mechRoom,Meter,31,METER,meter,None,NaN,None,5,None,None,,,[],[],"[{'id': 206165, 'building_id': 230, 'last_upda...",[meter]
3,19358,230,meter-ac1-switchGearRoom,meter-ac1-switchGearRoom,Meter,31,METER,meter,None,NaN,None,5,None,None,,,[],[],"[{'id': 206166, 'building_id': 230, 'last_upda...",[meter]
4,19359,230,meter-ac3-lobby,meter-ac3-lobby,Meter,31,METER,meter,None,NaN,None,5,None,None,,,[],[],"[{'id': 206167, 'building_id': 230, 'last_upda...",[meter]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,19441,230,meter-uh1-mechRoom,meter-uh1-mechRoom,Meter,31,METER,meter,None,NaN,None,5,None,None,,,[],[],"[{'id': 206256, 'building_id': 230, 'last_upda...",[meter]
87,19442,230,meter-uh2-mechRoom,meter-uh2-mechRoom,Meter,31,METER,meter,None,NaN,None,5,None,None,,,[],[],"[{'id': 206257, 'building_id': 230, 'last_upda...",[meter]
88,19443,230,meter-vacuumPump-switchGearRoom,meter-vacuumPump-switchGearRoom,Meter,31,METER,meter,None,NaN,None,5,None,None,,,[],[],"[{'id': 206258, 'building_id': 230, 'last_upda...",[meter]
89,19444,230,meter-windowWash-mechRoom,meter-windowWash-mechRoom,Meter,31,METER,meter,None,NaN,None,5,None,None,,Window Wash Rig(Mechanical Rm)_Active Power(kW),[],[],"[{'id': 206259, 'building_id': 230, 'last_upda...",[meter]


In [29]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_230_points = []
        for line in j:
            if line['building_id'] == 230:
                b_230_points.append(line)

df_b_230_points = pd.DataFrame(b_230_points).set_index('id')
df_b_230_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
206084,230,1.567396e+12,1.551416e+12,,None,,,Duplex Air Compressor(Switchgear Rm)_Total Cos...,,None,23,0.00,Virtual,434,18,eb7efcd46e79ae110f21a02015922b49,nyserda+0000111479+Site-Virtual+Duplex Air Com...,None,19355
206122,230,1.567396e+12,1.551416e+12,,None,,,PP-1 Left Section Brk 10(Switchgear Rm)_Total ...,,None,23,304.0,Virtual,434,18,e7a16b55b8f82216814f038ef633dacc,nyserda+0000111479+Site-Virtual+PP-1 Left Sect...,None,19355
206075,230,1.567396e+12,1.551416e+12,,None,,,ACC-1(Switchgear Rm)_Total Cost ($),,None,23,0.18,Virtual,434,18,04f37768540e479740bc5e4c4f9bb546,nyserda+0000111479+Site-Virtual+ACC-1(Switchge...,None,19355
206078,230,1.567396e+12,1.551416e+12,,None,,,Condensate Pump(Mechanical Rm)_Total Cost ($),,None,23,0.00,Virtual,434,18,7fdd3388d0d43503dce8356421282987,nyserda+0000111479+Site-Virtual+Condensate Pum...,None,19355
206089,230,1.567396e+12,1.551416e+12,,None,,,Electric Heater 2(Mechanical Rm)_Total Cost ($),,None,23,104.0,Virtual,434,18,0a7bbbd72563d689f393a5578cb431fb,nyserda+0000111479+Site-Virtual+Electric Heate...,None,19355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206256,230,1.567396e+12,1.551416e+12,,None,,,Electric Heater 1(Mechanical Rm)_Active Power(kW),kW,None,17,0.00,Real Power,221,13,d8ba59c0661c5dbd39908fc4835348df,nyserda+0000111479+meter-uh1-mechRoom+Electric...,None,19441
206257,230,1.567396e+12,1.551416e+12,,None,,,Electric Heater 2(Mechanical Rm)_Active Power(kW),kW,None,17,0.00,Real Power,221,13,643f474723339b42c4ece367e0ed16e0,nyserda+0000111479+meter-uh2-mechRoom+Electric...,None,19442
206258,230,1.567396e+12,1.551416e+12,,None,,,Duplex Vacuum Pumps(Switchgear Rm)_Active Powe...,kW,None,17,0.01,Real Power,221,13,3407b8742d7676ab94befb47a6c4b7dc,nyserda+0000111479+meter-vacuumPump-switchGear...,None,19443


# Building 423:
- Seems to have water meters for the baths and kitchen sinks?
- Possibly a hotel, or at least multi-residential, given that there are Zone Temperature datapoints for every
apartment
 and penthouse.
- Hot and Cold Water Meters for sink and bath
- M.E.R could stand for Main Equipment Room
- Electricity and Gas supply seems to be centralised

In [30]:
if use_local:
    with open('../api/buildings/423.json', 'r') as f:
        j = json.load(f)
        b_423 = pd.json_normalize(j)
else:
    b_423 = client.get_building_equipment(423)
b_423

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,26597,423,site-APT-PH3,APT-PH3,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 280771, 'building_id': 423, 'last_upda...",[site]
1,26598,423,site-APT-PH4,APT-PH4,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 280774, 'building_id': 423, 'last_upda...",[site]
2,26599,423,site-APT-PH5,APT-PH5,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 280778, 'building_id': 423, 'last_upda...",[site]
3,26600,423,site-APT-PH6,APT-PH6,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 280779, 'building_id': 423, 'last_upda...",[site]
4,26601,423,site-APT-PH8,APT-PH8,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 280782, 'building_id': 423, 'last_upda...",[site]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,26731,423,hotWaterPump-1,1,Pump,32,PUMP,pump,Hot Water Pump,25.0,hotWaterPump,1,None,None,None,None,[],[],"[{'id': 280365, 'building_id': 423, 'last_upda...","[pump, hvac, hotWaterPump]"
135,26732,423,hotWaterPump-2,2,Pump,32,PUMP,pump,Hot Water Pump,25.0,hotWaterPump,1,None,None,None,None,[],[],"[{'id': 280366, 'building_id': 423, 'last_upda...","[pump, hvac, hotWaterPump]"
136,26733,423,pump-3,3,Pump,32,PUMP,pump,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 280369, 'building_id': 423, 'last_upda...","[pump, hvac]"
137,26734,423,site-APT-PH1,APT-PH1,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 280766, 'building_id': 423, 'last_upda...",[site]


In [31]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_423_points = []
        for line in j:
            if line['building_id'] == 423:
                b_423_points.append(line)

df_b_423_points = pd.DataFrame(b_423_points).set_index('id')
df_b_423_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
280771,423,1.606798e+12,1.575180e+12,None,None,None,Apt PH-3 Zone-BR2 Zone Temperature,None,F,f,2,71.59,Zone Temperature,77,1,e89810ee95fed44df9f3b11d3da8db54,nyserda+124540+site-APT-PH3+Apt PH-3 Zone-BR2 ...,None,26597
280770,423,1.606798e+12,1.575179e+12,None,None,None,Apt PH-3 Zone-BR1 Zone Temperature,None,F,f,2,73.40,Zone Temperature,77,1,5ab09836cb322b6385a3352f7e84ad4f,nyserda+124540+site-APT-PH3+Apt PH-3 Zone-BR1 ...,None,26597
280772,423,1.606798e+12,1.575180e+12,None,None,None,Apt PH-3 Zone-LR Zone Temperature,None,F,f,2,73.40,Zone Temperature,77,1,564d2aae2c6d8a144b183c36f7162635,nyserda+124540+site-APT-PH3+Apt PH-3 Zone-LR Z...,None,26597
280774,423,1.606798e+12,1.575183e+12,None,None,None,Apt PH-4 Zone-BR2 Zone Temperature,None,F,f,2,75.19,Zone Temperature,77,1,7eb371c31665b3f2e9b73bdb0314c757,nyserda+124540+site-APT-PH4+Apt PH-4 Zone-BR2 ...,None,26598
280775,423,1.606798e+12,1.575179e+12,None,None,None,Apt PH-4 Zone-LR Zone Temperature,None,F,f,2,75.19,Zone Temperature,77,1,8a58d71c5ccc1ad4bc0406a8004e2c1a,nyserda+124540+site-APT-PH4+Apt PH-4 Zone-LR Z...,None,26598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280766,423,1.606798e+12,1.575179e+12,None,None,None,Apt PH-1 Zone-LR Zone Temperature,None,F,f,2,71.59,Zone Temperature,77,1,e28e1c004af6af5762475eafa4d17085,nyserda+124540+site-APT-PH1+Apt PH-1 Zone-LR Z...,None,26734
280765,423,1.606798e+12,1.575179e+12,None,None,None,Apt PH-1 Zone-BR1 Zone Temperature,None,F,f,2,69.80,Zone Temperature,77,1,6aa48865ab30f0bd7316680a91421f6e,nyserda+124540+site-APT-PH1+Apt PH-1 Zone-BR1 ...,None,26734
280769,423,1.606798e+12,1.575179e+12,None,None,None,Apt PH-2 Zone-LR Zone Temperature,None,F,f,2,73.40,Zone Temperature,77,1,4611e8acdfd47ffe8fe696750a2c8357,nyserda+124540+site-APT-PH2+Apt PH-2 Zone-LR Z...,None,26735


# Building 426: 480 W 42nd St Signature Theatre
- Seems to have separate datasets for each space.
- Spaces are labelled as WSHP1, WP1 etc.
- The signature theatre has 5 performance spaces and a set of offices (?)

In [32]:
all_buildings = client.get_all_buildings()
df = pd.json_normalize(all_buildings)
df

,id,org_id,name,address,sq_ft,image_src,bms_manufacturer,bms_product_name,bms_version,timezone,...,info.sunend,info.geoCity,info.geoState,info.m2fstart,info.satstart,info.sunstart,info.yearBuilt,info.geoCountry,info.weatherRef,info.customerType
0,441,5,88737,None,551143.0,None,None,None,None,America/New_York,...,,Bronx,NY,,,,,US,,Multifamily
1,140,5,83483,None,21869.0,None,None,None,None,America/New_York,...,,New York,NY,,,,,US,,Commercial Office
2,191,5,94414,None,198057.0,None,None,None,None,America/New_York,...,23:00,Victor,NY,07:30,07:30,07:30,,US,,Commercial Retail
3,231,5,89496,None,NaN,None,None,None,None,America/New_York,...,,New York,NY,,,,,US,,
4,248,5,116742,None,127000.0,None,None,None,None,America/New_York,...,,New York,NY,,,,,US,,Multifamily
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,353,5,94413,None,198057.0,None,None,None,None,America/New_York,...,23:00,Henrietta,NY,07:30,07:30,07:30,,US,,Commercial Retail
225,370,5,95457,None,421316.0,None,None,None,None,America/New_York,...,,New York,NY,,,,,US,,Hospitality
226,383,5,79528,None,75000.0,None,None,None,None,America/New_York,...,NaN,Purchase,New York,04:00:00,NaN,NaN,1971,US,NaN,Commercial Office
227,398,5,109188,None,860000.0,None,None,None,None,America/New_York,...,,New York,NY,,,,,US,,Commercial Office


In [33]:
if use_local:
    with open('../api/buildings/426.json', 'r') as f:
        j = json.load(f)
        b_426 = pd.json_normalize(j)
else:
    b_426 = client.get_building_equipment(426)
b_426

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,26935,426,chilledWaterPlant,chilledWaterPlant,Chilled Water Plant,20,CHWS,chilledWaterPlant,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 283189, 'building_id': 426, 'last_upda...","[hvac, chilledWaterPlant]"
1,26936,426,chilledWaterPump1,chilledWaterPump1,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[],[],"[{'id': 283195, 'building_id': 426, 'last_upda...","[pump, hvac, chilledWaterPump]"
2,26937,426,chilledWaterPump2,chilledWaterPump2,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[],[],"[{'id': 283198, 'building_id': 426, 'last_upda...","[pump, hvac, chilledWaterPump]"
3,26938,426,domesticHotWaterPlant,domesticHotWaterPlant,Hot Water Plant,30,HWS,hotWaterPlant,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 283200, 'building_id': 426, 'last_upda...","[hvac, hotWaterPlant]"
4,26939,426,fan5-exhaust,fan5-exhaust,Fan,26,FAN,fan,Exhaust Fan,12.0,exhaustFan,4,None,None,None,None,[],[],"[{'id': 283202, 'building_id': 426, 'last_upda...","[fan, hvac, exhaustFan]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,27022,426,vav50,vav50,Variable Air Volume,7,VAV,vav,None,NaN,None,3,None,None,None,None,[],[],"[{'id': 284073, 'building_id': 426, 'last_upda...","[vav, hvac]"
88,27023,426,vav51,vav51,Variable Air Volume,7,VAV,vav,None,NaN,None,3,None,None,None,None,[],[],"[{'id': 284075, 'building_id': 426, 'last_upda...","[vav, hvac]"
89,27024,426,vav52,vav52,Variable Air Volume,7,VAV,vav,None,NaN,None,3,None,None,None,None,[],[],"[{'id': 284080, 'building_id': 426, 'last_upda...","[vav, hvac]"
90,27025,426,vav53,vav53,Variable Air Volume,7,VAV,vav,None,NaN,None,3,None,None,None,None,[],[],"[{'id': 284084, 'building_id': 426, 'last_upda...","[vav, hvac]"


In [34]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_426_points = []
        for line in j:
            if line['building_id'] == 426:
                b_426_points.append(line)

df_b_426_points = pd.DataFrame(b_426_points).set_index('id')
df_b_426_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
283191,426,1.606798e+12,1.512104e+12,None,None,None,480 W 42nd St - Signature Theatre CwPlant CWRTemp,None,F,f,2,91.01,Condenser Water Return Temperature,388,1,56941ee5de2c587f311418d74b481ded,nyserda+000076513+condenserWaterPlant+480 W 42...,None,26935
283190,426,1.606798e+12,1.512104e+12,None,None,None,480 W 42nd St - Signature Theatre CwPlant CWFlow,None,gpm,gpm,8,528.07,Condenser Water Supply Flow,390,5,bfc3c79edb52cc681e94d4d0c6779eea,nyserda+000076513+condenserWaterPlant+480 W 42...,None,26935
283187,426,1.606798e+12,1.512104e+12,None,None,None,480 W 42nd St - Signature Theatre CwPlant CW B...,None,%,%,3,0.0,Condenser Water Bypass Valve Position,380,7,8a32f9d74ad15fabeaf8a84c1a4866b0,nyserda+000076513+condenserWaterPlant+480 W 42...,None,26935
283192,426,1.606798e+12,1.512104e+12,None,None,None,480 W 42nd St - Signature Theatre CwPlant CWSTemp,None,F,f,2,89.15,Condenser Water Supply Temperature,392,1,41046ec6a9a1318fdbf63cfdaa08019b,nyserda+000076513+condenserWaterPlant+480 W 42...,None,26935
283186,426,1.606798e+12,1.512104e+12,None,None,None,480 W 42nd St - Signature Theatre CwPlant CWBTU,None,BTU,btu,14,486593.5,Electric Consumption,178,14,59934b7d43024bd4292989f5c1b86d4c,nyserda+000076513+condenserWaterPlant+480 W 42...,None,26935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284092,426,1.606797e+12,1.512104e+12,None,None,None,480 W 42nd St - Signature Theatre VAV54 SATemp,None,F,f,2,73.68,Discharge Air Temperature,79,1,98311c195735d14d30cceb08845fd8a3,nyserda+000076513+vav54+480 W 42nd St - Signat...,None,27026
284091,426,1.606798e+12,1.512104e+12,None,None,None,480 W 42nd St - Signature Theatre VAV54 OA Air...,None,cfm,cfm,7,0.0,Outside Air Flow Setpoint,140,4,1057abca8db4255d0a3915520b040e7f,nyserda+000076513+vav54+480 W 42nd St - Signat...,None,27026
284089,426,1.606798e+12,1.539096e+12,None,None,None,480 W 42nd St - Signature Theatre VAV54 Airflow,None,cfm,cfm,7,0.0,Discharge Air Flow,112,4,d44de7865bf33da89e7c371555c7ca56,nyserda+000076513+vav54+480 W 42nd St - Signat...,None,27026


# Building 438: AOL Time Warner Building

In [35]:
if use_local:
    with open('../api/buildings/438.json', 'r') as f:
        j = json.load(f)
        b_438 = pd.json_normalize(j)
else:
    b_438 = client.get_building_equipment(438)
b_438

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,28550,438,elecMeterLoad-053_SM-020,053_SM-020,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 308715, 'building_id': 438, 'last_upda...","[meter, elecMeter]"
1,28551,438,elecMeterLoad-053_WM-014,053_WM-014,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 308716, 'building_id': 438, 'last_upda...","[meter, elecMeter]"
2,28552,438,ahu-37N-1,37N-1,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[],"[{'id': 308008, 'building_id': 438, 'last_upda...","[ahu, hvac]"
3,28553,438,ahu-14N-1,14N-1,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[],"[{'id': 307611, 'building_id': 438, 'last_upda...","[ahu, hvac]"
4,28554,438,ahu-15N-1,15N-1,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],[],"[{'id': 307615, 'building_id': 438, 'last_upda...","[ahu, hvac]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,28766,438,elecMeterLoad-046_SM-019,046_SM-019,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 308710, 'building_id': 438, 'last_upda...","[meter, elecMeter]"
217,28767,438,elecMeterLoad-050_BM-381,050_BM-381,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 308711, 'building_id': 438, 'last_upda...","[meter, elecMeter]"
218,28768,438,elecMeterLoad-050_BM-382,050_BM-382,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 308712, 'building_id': 438, 'last_upda...","[meter, elecMeter]"
219,28769,438,elecMeterLoad-052_BM-0CT,052_BM-0CT,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 308713, 'building_id': 438, 'last_upda...","[meter, elecMeter]"


In [36]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_438_points = []
        for line in j:
            if line['building_id'] == 438:
                b_438_points.append(line)

df_b_438_points = pd.DataFrame(b_438_points).set_index('id')
df_b_438_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
308715,438,1.604117e+12,1.588388e+12,None,None,None,AOLTW.FPMETER-053.SM-020,None,kWh,kwh,16,32400.0,Electric Consumption,178,14,c4309e3e3a62a11d8109e4ce35c80d31,nyserda+0000116350+elecMeterLoad-053_SM-020+AO...,None,28550
308716,438,1.604117e+12,1.588388e+12,None,None,None,AOLTW.FPMETER-053.WM-014,None,kWh,kwh,16,4930.0,Electric Consumption,178,14,22d65095d247827886d6d3aa82c5f919,nyserda+0000116350+elecMeterLoad-053_WM-014+AO...,None,28551
308008,438,1.604189e+12,1.588306e+12,None,None,None,Base Building AHU-37N-1_Final Filter Pressure ...,None,On/Off,on/off,20,0.0,Filter Status,331,11,ecb82941d1926a305671d82c8787229f,nyserda+0000116350+ahu-37N-1+Base Building AHU...,None,28552
308007,438,1.604189e+12,1.588306e+12,None,None,None,Base Building AHU-37N-1_Cooling Coil Valve Com...,None,%,%,3,0.0,Cooling Coil Valve Position,239,7,a86406b92c0b803d965c784436a20d5c,nyserda+0000116350+ahu-37N-1+Base Building AHU...,None,28552
308009,438,1.604189e+12,1.588306e+12,None,None,None,Base Building AHU-37N-1_Occupied Mode,None,Occ,occ,18,0.0,Occupancy Status,177,10,fe5df5fc1f87de13b082790924217def,nyserda+0000116350+ahu-37N-1+Base Building AHU...,None,28552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308710,438,1.604117e+12,1.588388e+12,None,None,None,AOLTW.FPMETER-046.SM-019,None,kWh,kwh,16,200.0,Electric Consumption,178,14,6b642d768b319fa54824c2f0acea88c1,nyserda+0000116350+elecMeterLoad-046_SM-019+AO...,None,28766
308711,438,1.604117e+12,1.588388e+12,None,None,None,AOLTW.FPMETER-050.BM-381,None,kWh,kwh,16,310.0,Electric Consumption,178,14,2ff9694db59824e93d8dffdc90e7963f,nyserda+0000116350+elecMeterLoad-050_BM-381+AO...,None,28767
308712,438,1.604117e+12,1.588388e+12,None,None,None,AOLTW.FPMETER-050.BM-382,None,kWh,kwh,16,130.0,Electric Consumption,178,14,0db505f0a8d72846e0e5f4900b6f7547,nyserda+0000116350+elecMeterLoad-050_BM-382+AO...,None,28768


# Building 440: Floor Level Power Usage
- Occupancy value
- Commercial Office Building

In [37]:
if use_local:
    with open('../api/buildings/440.json', 'r') as f:
        j = json.load(f)
        b_440 = pd.json_normalize(j)
else:
    b_440 = client.get_building_equipment(440)
b_440

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,28796,440,virtual,None,Virtual,46,VIRT,virtual,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 310007, 'building_id': 440, 'last_upda...",[]
1,33331,440,elecMeter-10,10,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 309978, 'building_id': 440, 'last_upda...","[meter, elecMeter]"
2,33332,440,elecMeter-16,16,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 309979, 'building_id': 440, 'last_upda...","[meter, elecMeter]"
3,33333,440,elecMeter-17,17,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 309980, 'building_id': 440, 'last_upda...","[meter, elecMeter]"
4,33334,440,elecMeter-18,18,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 309981, 'building_id': 440, 'last_upda...","[meter, elecMeter]"
5,33335,440,elecMeter-19,19,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 309982, 'building_id': 440, 'last_upda...","[meter, elecMeter]"
6,33336,440,elecMeter-20,20,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 309983, 'building_id': 440, 'last_upda...","[meter, elecMeter]"
7,33337,440,elecMeter-21,21,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 309984, 'building_id': 440, 'last_upda...","[meter, elecMeter]"
8,33338,440,elecMeter-2,2,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 309970, 'building_id': 440, 'last_upda...","[meter, elecMeter]"
9,33339,440,elecMeter-23,23,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 309985, 'building_id': 440, 'last_upda...","[meter, elecMeter]"


In [38]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_440_points = []
        for line in j:
            if line['building_id'] == 440:
                b_440_points.append(line)

df_b_440_points = pd.DataFrame(b_440_points).set_index('id')
df_b_440_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
310007,440,1.633147e+12,1.601615e+12,None,None,None,Total Occupancy value,None,?,?,23,6.0,Virtual,434,18,17eb6130fe4b2edd2333f37511259c3b,nyserda+0000093400+virtual+Total Occupancy value,None,28796
310006,440,1.633147e+12,1.601615e+12,None,None,None,Total Electric Demand value,None,?,?,23,229.78,Virtual,434,18,01355fd2a798ed2539b46b15df49ba5e,nyserda+0000093400+virtual+Total Electric Dema...,None,28796
310005,440,1.633147e+12,1.601615e+12,None,None,None,Occupancy 9th Floor value,None,?,?,23,0.0,Virtual,434,18,55e285e8ab60eeace3f9ed812cbdc907,nyserda+0000093400+virtual+Occupancy 9th Floor...,None,28796
310004,440,1.633147e+12,1.601615e+12,None,None,None,Occupancy 8th Floor value,None,?,?,23,4.0,Virtual,434,18,37fc70d70715980cbef5edce4d03b717,nyserda+0000093400+virtual+Occupancy 8th Floor...,None,28796
310003,440,1.633147e+12,1.601615e+12,None,None,None,Occupancy 7th Floor value,None,?,?,23,2.0,Virtual,434,18,171a09d5c95beff32d9ede40698073bd,nyserda+0000093400+virtual+Occupancy 7th Floor...,None,28796
310002,440,1.633147e+12,1.601615e+12,None,None,None,Occupancy 6th Floor value,None,?,?,23,0.0,Virtual,434,18,bf9e4cbdd56f180bbe99781bd987ec19,nyserda+0000093400+virtual+Occupancy 6th Floor...,None,28796
310001,440,1.633147e+12,1.601615e+12,None,None,None,Occupancy 5th Floor value,None,?,?,23,0.0,Virtual,434,18,c7b9101418b74f3851b6095ec633e401,nyserda+0000093400+virtual+Occupancy 5th Floor...,None,28796
310000,440,1.633147e+12,1.601615e+12,None,None,None,Occupancy 4th Floor value,None,?,?,23,0.0,Virtual,434,18,3e33f107a54316626d926c7c133981a9,nyserda+0000093400+virtual+Occupancy 4th Floor...,None,28796
309999,440,1.633147e+12,1.601615e+12,None,None,None,Occupancy 3rd Floor value,None,?,?,23,0.0,Virtual,434,18,4daf06646dc2df7911c043f9fa49f53f,nyserda+0000093400+virtual+Occupancy 3rd Floor...,None,28796


# Building 503:

In [39]:
if use_local:
    with open('../api/buildings/503.json', 'r') as f:
        j = json.load(f)
        b_503 = pd.json_normalize(j)
else:
    b_503 = client.get_building_equipment(503)
b_503

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,33941,503,ahu-27,27,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,None,None,[],"[33957, 33958, 33959, 33960]","[{'id': 387465, 'building_id': 503, 'last_upda...","[ahu, hvac]"
1,33942,503,chilledWaterPump-4A,4A,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[],[],"[{'id': 387466, 'building_id': 503, 'last_upda...","[pump, hvac, chilledWaterPump]"
2,33943,503,chilledWaterPump-4B,4B,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[],[],"[{'id': 387467, 'building_id': 503, 'last_upda...","[pump, hvac, chilledWaterPump]"
3,33944,503,chilledWaterPump-P-4A,P-4A,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[],[],"[{'id': 387468, 'building_id': 503, 'last_upda...","[pump, hvac, chilledWaterPump]"
4,33945,503,chilledWaterPump-P-4B,P-4B,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[],[],"[{'id': 387469, 'building_id': 503, 'last_upda...","[pump, hvac, chilledWaterPump]"
5,33946,503,chilledWaterPump-SWP-5,SWP-5,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[],[],"[{'id': 387470, 'building_id': 503, 'last_upda...","[pump, hvac, chilledWaterPump]"
6,33947,503,chilledWaterPump-SWP-6,SWP-6,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[],[],"[{'id': 387471, 'building_id': 503, 'last_upda...","[pump, hvac, chilledWaterPump]"
7,33948,503,chiller-1,1,Chiller,21,CH,chiller,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 387475, 'building_id': 503, 'last_upda...","[chiller, hvac]"
8,33949,503,chiller-1-Primary,1-Primary,Chiller,21,CH,chiller,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 387477, 'building_id': 503, 'last_upda...","[chiller, hvac]"
9,33950,503,chiller-2,2,Chiller,21,CH,chiller,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 387481, 'building_id': 503, 'last_upda...","[chiller, hvac]"


In [40]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_503_points = []
        for line in j:
            if line['building_id'] == 503:
                b_503_points.append(line)

df_b_503_points = pd.DataFrame(b_503_points).set_index('id')
df_b_503_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
387465,503,1.622447e+12,1.606730e+12,None,None,None,madisonChilledWaterSupplyMeasurement_27th Floo...,None,None,f,2,67.20,Chilled Water Supply Temperature,89,1,e6a4cb83155427da73005dcacbc2c512,nyserda+0000145813+ahu-27+madisonChilledWaterS...,None,33941
387464,503,1.606712e+12,1.575176e+12,None,None,None,MadisonChilledWaterSupplyMeasurement()_27th Fl...,None,None,f,2,67.09,Chilled Water Supply Temperature,89,1,99a0934f4a76b35c93c18ebeecfc567e,nyserda+0000145813+ahu-27+MadisonChilledWaterS...,None,33941
387466,503,1.622447e+12,1.575176e+12,None,None,None,madisonChilledWaterPumpSpeedMeasurement_Cellar...,None,None,rpm,29,50.0,Speed Sensor,636,17,59ec4012cdd19e74fdda929de8592b01,nyserda+0000145813+chilledWaterPump-4A+madison...,None,33942
387467,503,1.622447e+12,1.575176e+12,None,None,None,madisonChilledWaterPumpSpeedMeasurement_Cellar...,None,None,rpm,29,0.0,Speed Sensor,636,17,334047aa7b9541829d8d5d7a7e2aabd4,nyserda+0000145813+chilledWaterPump-4B+madison...,None,33943
387468,503,1.622447e+12,1.575176e+12,None,None,None,madisonChilledWaterPumpSpeedMeasurement_Cellar...,None,None,%,3,100.0,Speed Sensor,636,17,2609e24a0049e0cbf9ccdda946568938,nyserda+0000145813+chilledWaterPump-P-4A+madis...,None,33944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387616,503,1.622447e+12,1.575176e+12,None,None,None,madisonSupplyAirFanSpeedMeasurement_7th Floor:...,None,None,rpm,29,0.0,Speed Sensor,636,17,d8acf216ccda93b94bfec39f9e890b6d,nyserda+0000145813+supplyFan-7-F-3+madisonSupp...,None,33972
387615,503,1.606712e+12,1.575176e+12,None,None,None,MadisonSupplyAirFanSpeedMeasurement()_7th Floo...,None,None,rpm,29,0.0,Speed Sensor,636,17,af249cd0b6ca3d0ed05f71fe4a23282b,nyserda+0000145813+supplyFan-7-F-3+MadisonSupp...,None,33972
387618,503,1.622447e+12,1.575176e+12,None,None,None,madisonSupplyFanStatusMeasurement_Cellar Floor...,None,None,on/off,20,0.0,Status,121,11,44237010a1f1cf8a7d50eb7c6b2c1de5,nyserda+0000145813+supplyFan-Cellar-ACU+madiso...,None,33973


# Building 141: Cost + Usage Data for Plant Equipment
- Has 25 Elevators

In [41]:
if use_local:
    with open('../api/buildings/141.json', 'r') as f:
        j = json.load(f)
        b_141 = pd.json_normalize(j)
else:
    b_141 = client.get_building_equipment(141)
b_141

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,18105,141,chilledWaterPump-lobby,chilledWaterPump-lobby,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,,Lobby Chiller/Pump,[],[],"[{'id': 198473, 'building_id': 141, 'last_upda...","[pump, hvac, chilledWaterPump]"
1,18106,141,chilledWaterPump36-2,chilledWaterPump36-2,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,,Pump 36-2 (CHW),[],[],"[{'id': 198476, 'building_id': 141, 'last_upda...","[pump, hvac, chilledWaterPump]"
2,18107,141,chilledWaterPump5,chilledWaterPump5,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,,Pump 5 (Chilled Water),[],[],"[{'id': 198478, 'building_id': 141, 'last_upda...","[pump, hvac, chilledWaterPump]"
3,18108,141,chilledWaterPump6,chilledWaterPump6,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,,Pump 6 (Chilled Water),[],[],"[{'id': 198480, 'building_id': 141, 'last_upda...","[pump, hvac, chilledWaterPump]"
4,18109,141,chiller3,chiller3,Chiller,21,CH,chiller,None,NaN,None,1,None,None,,Chiller 3,[],[],"[{'id': 198482, 'building_id': 141, 'last_upda...","[chiller, hvac]"
5,18110,141,condenserWaterPump-secondary,condenserWaterPump-secondary,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,,Secondary Condenser Water Pumps 28/29,[],[],"[{'id': 198484, 'building_id': 141, 'last_upda...","[pump, hvac, condenserWaterPump]"
6,18111,141,condenserWaterPump36-1,condenserWaterPump36-1,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,,Pump 36-1 (CDW),[],[],"[{'id': 198486, 'building_id': 141, 'last_upda...","[pump, hvac, condenserWaterPump]"
7,18112,141,condenserWaterPump8,condenserWaterPump8,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,,Pump 8 (Condenser Water),[],[],"[{'id': 198488, 'building_id': 141, 'last_upda...","[pump, hvac, condenserWaterPump]"
8,18113,141,condenserWaterPump9,condenserWaterPump9,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,,Pump 9 (Condenser Water),[],[],"[{'id': 198489, 'building_id': 141, 'last_upda...","[pump, hvac, condenserWaterPump]"
9,18114,141,elecPanel-lvp,elecPanel-lvp,Electrical Panel,24,ELEC,panel,None,NaN,None,1,None,None,,UP-12UC & 12UD LVP-12-(1-3),[],[],"[{'id': 198492, 'building_id': 141, 'last_upda...",[panel]


In [42]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_141_points = []
        for line in j:
            if line['building_id'] == 141:
                b_141_points.append(line)

df_b_141_points = pd.DataFrame(b_141_points).set_index('id')
df_b_141_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
198473,141,1.552020e+12,1.520399e+12,,None,,,Lobby Chiller/Pump Total Cost ($),,None,23,0.13,Virtual,434,18,69ab5fbf6c0ac735343ef796ea71e05b,nyserda+0000084481+chilledWaterPump-lobby+Lobb...,None,18105
198474,141,1.552020e+12,1.520399e+12,,None,,,Lobby Chiller/Pump Active Power(kW),kW,None,17,2.94,Real Power,221,13,d97fdf24df7e6d0977888cd0ecbc7ecc,nyserda+0000084481+chilledWaterPump-lobby+Lobb...,None,18105
198476,141,1.552020e+12,1.520399e+12,,None,,,Pump 36-2 (CHW) Active Power(kW),kW,None,17,20.85,Real Power,221,13,88b7faa51880632556e75b9983829ba5,nyserda+0000084481+chilledWaterPump36-2+Pump 3...,None,18106
198475,141,1.552020e+12,1.520399e+12,,None,,,Pump 36-2 (CHW) Total Cost ($),,None,23,0.93,Virtual,434,18,2aeb8562ee63924cda408e015904f008,nyserda+0000084481+chilledWaterPump36-2+Pump 3...,None,18106
198478,141,1.552020e+12,1.520399e+12,,None,,,Pump 5 (Chilled Water) Active Power(kW),kW,None,17,0.0,Real Power,221,13,6e0892ba6d2c2b6b8caab5ad2a59b667,nyserda+0000084481+chilledWaterPump5+Pump 5 (C...,None,18107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198550,141,1.552020e+12,1.520399e+12,,None,,,TP 1-4 (Tenant Pumps) Active Power(kW),kW,None,17,13.78,Real Power,221,13,8f67e164ff408d5086e698edb925302f,nyserda+0000084481+pump-tenant+TP 1-4 (Tenant ...,None,18143
198552,141,1.552020e+12,1.520399e+12,,None,,,Pump 7 (Standby Pump) Active Power(kW),kW,None,17,0.0,Real Power,221,13,51be8b2b97a6eeacd31cf7607cddd682,nyserda+0000084481+pump7-standby+Pump 7 (Stand...,None,18144
198551,141,1.552020e+12,1.520399e+12,,None,,,Pump 7 (Standby Pump) Total Cost ($),,None,23,0.0,Virtual,434,18,7689a39fe518e88cd8d5e17ad48ae6ae,nyserda+0000084481+pump7-standby+Pump 7 (Stand...,None,18144


# Building 118: Assorted Plug Loads

In [43]:
if use_local:
    with open('../api/buildings/118.json', 'r') as f:
        j = json.load(f)
        b_118 = pd.json_normalize(j)
else:
    b_118 = client.get_building_equipment(118)
b_118

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,17436,118,ac-office,ac-office,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,A.C (Office)(Office),[],[],"[{'id': 194441, 'building_id': 118, 'last_upda...","[ahu, hvac]"
1,17437,118,ac1-hallwayBasement,ac1-hallwayBasement,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,A.C-1 (Basement)(Hallway),[],[],"[{'id': 194443, 'building_id': 118, 'last_upda...","[ahu, hvac]"
2,17438,118,ac1-hallwayFloor6,ac1-hallwayFloor6,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,A.C-1 (6th Floor)(Hallway),[],[],"[{'id': 194445, 'building_id': 118, 'last_upda...","[ahu, hvac]"
3,17439,118,ac1-hallwayFloor8,ac1-hallwayFloor8,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,A.C-1 (8th Floor)(Hallway),[],[],"[{'id': 194447, 'building_id': 118, 'last_upda...","[ahu, hvac]"
4,17440,118,ac10-hallwayFloor10NE,ac10-hallwayFloor10NE,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,A.C-NE (10th Floor)(Hallway),[],[],"[{'id': 194448, 'building_id': 118, 'last_upda...","[ahu, hvac]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,17553,118,pump-p2b-boilerRoom1,pump-p2b-boilerRoom1,Pump,32,PUMP,pump,None,NaN,None,1,None,None,,Pump-P2.B(Boiler Rm)(1),[],[],"[{'id': 194685, 'building_id': 118, 'last_upda...","[pump, hvac]"
118,17554,118,pump1,pump1,Pump,32,PUMP,pump,None,NaN,None,1,None,None,,Circulator Pump(Hallway)(1),[],[],"[{'id': 194687, 'building_id': 118, 'last_upda...","[pump, hvac]"
119,17555,118,pump1-boilerRoom,pump1-boilerRoom,Pump,32,PUMP,pump,None,NaN,None,1,None,None,,Pump-1(Boiler Rm),[],[],"[{'id': 194690, 'building_id': 118, 'last_upda...","[pump, hvac]"
120,17556,118,pump1-boilerRoom1,pump1-boilerRoom1,Pump,32,PUMP,pump,None,NaN,None,1,None,None,,Pump-1(Boiler Rm)(1),[],[],"[{'id': 194692, 'building_id': 118, 'last_upda...","[pump, hvac]"


In [44]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_118_points = []
        for line in j:
            if line['building_id'] == 118:
                b_118_points.append(line)

df_b_118_points = pd.DataFrame(b_118_points).set_index('id')
df_b_118_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
194441,118,1.525319e+12,1.493698e+12,,None,,,A.C (Office)(Office)_A/C (Office) Total Cost ($),,None,23,0.73,Virtual,434,18,1e1c7303a30671d48c7d80a8c06ad925,nyserda+0000073107+ac-office+A.C (Office)(Offi...,None,17436
194440,118,1.525319e+12,1.493698e+12,,None,,,A.C (Office)(Office)_A/C (Office) Active Power...,kW,None,17,16.43,Real Power,221,13,85a9c9091261307380eb6efebf62fd7a,nyserda+0000073107+ac-office+A.C (Office)(Offi...,None,17436
194442,118,1.525319e+12,1.493698e+12,,None,,,A.C-1 (Basement)(Hallway)_A/C-1 (Basement) Act...,kW,None,17,0.24,Real Power,221,13,daf0d0ac52d65b4ce8c0d40240ec5c25,nyserda+0000073107+ac1-hallwayBasement+A.C-1 (...,None,17437
194443,118,1.525319e+12,1.493698e+12,,None,,,A.C-1 (Basement)(Hallway)_A/C-1 (Basement) Tot...,,None,23,0.01,Virtual,434,18,555687e97f4449c451b9db9c67603f24,nyserda+0000073107+ac1-hallwayBasement+A.C-1 (...,None,17437
194445,118,1.525319e+12,1.493698e+12,,None,,,A.C-1 (6th Floor)(Hallway)_A/C-1 (6th Floor) T...,,None,23,0.0,Virtual,434,18,0eec3bdb5519d66d2db00a0806b30b39,nyserda+0000073107+ac1-hallwayFloor6+A.C-1 (6t...,None,17438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194693,118,1.514782e+12,1.498098e+12,,None,,,Pump-1(Boiler Rm)(1)_Pump-1 Total Cost ($),,None,23,204.0,Virtual,434,18,a0c0caeacd474a5e9adbed89d5706076,nyserda+0000073107+pump1-boilerRoom1+Pump-1(Bo...,None,17556
194694,118,1.525319e+12,1.498098e+12,,None,,,Water Tank Pump 1(Boiler Rm)_Water Tank Pump 1...,kW,None,17,0.0,Real Power,221,13,bfdd48892351a12e0c95b59188c097ba,nyserda+0000073107+waterTankPump1+Water Tank P...,None,17557
194695,118,1.498095e+12,1.493698e+12,,None,,,Water Tank Pump No. 1(Boiler Rm)_Water Tank Pu...,kW,None,17,0.0,Real Power,221,13,811444fb309a7709ca36f5b3e18c38c2,nyserda+0000073107+waterTankPump1+Water Tank P...,None,17557


# Building 128: 800 Grand Concourse

In [45]:
if use_local:
    with open('../api/buildings/128.json', 'r') as f:
        j = json.load(f)
        b_128 = pd.json_normalize(j)
else:
    b_128 = client.get_building_equipment(128)
b_128

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,17871,128,Site-Sensor,Site-Sensor,Site Sensors,37,SITE,site,None,None,None,5,None,None,,,[],[],"[{'id': 197377, 'building_id': 128, 'last_upda...",[site]
1,17872,128,Site-Virtual,Site-Virtual,Virtual,46,VIRT,virtual,None,None,None,5,None,None,,,[],[],"[{'id': 197391, 'building_id': 128, 'last_upda...",[]
2,17873,128,boiler1,boiler1,Boiler,19,BLR,boiler,None,None,None,1,None,None,,,[17874],[],"[{'id': 197393, 'building_id': 128, 'last_upda...","[boiler, hvac]"
3,17874,128,hotWaterPlant,hotWaterPlant,Hot Water Plant,30,HWS,hotWaterPlant,None,None,None,1,None,None,,,[],"[17873, 17875]","[{'id': 197400, 'building_id': 128, 'last_upda...","[hvac, hotWaterPlant]"
4,17875,128,boiler2,boiler2,Boiler,19,BLR,boiler,None,None,None,1,None,None,,,[17874],[],"[{'id': 197408, 'building_id': 128, 'last_upda...","[boiler, hvac]"


In [46]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_128_points = []
        for line in j:
            if line['building_id'] == 128:
                b_128_points.append(line)

df_b_128_points = pd.DataFrame(b_128_points).set_index('id')
df_b_128_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
197377,128,1.577854e+12,1.536553e+12,,None,,,800 Grand Concourse Apartment Sensors Apt LF NO,F,None,2,80.48,Zone Temperature,77,1,29e2b0e342b041eb099d32a5287c7e7f,nyserda+0000093623+Site-Sensor+800 Grand Conco...,None,17871
197375,128,1.577854e+12,1.550553e+12,,None,,,800 Grand Concourse Apartment Sensors Apt GL NO,F,None,2,67.32,Zone Temperature,77,1,55f4b4f99182a4a9f6e7491516d11300,nyserda+0000093623+Site-Sensor+800 Grand Conco...,None,17871
197364,128,1.577854e+12,1.536553e+12,,None,,,800 Grand Concourse Apartment Sensors Apt 6E SO,F,None,2,71.82,Zone Temperature,77,1,1e1b79c67e09d6fda2c6f1417b436f61,nyserda+0000093623+Site-Sensor+800 Grand Conco...,None,17871
197349,128,1.577854e+12,1.536553e+12,,None,,,800 Grand Concourse Apartment Sensors Apt 5N SO,F,None,2,74.75,Zone Temperature,77,1,57e2914e3d8401d0eaaa40779d4a6a97,nyserda+0000093623+Site-Sensor+800 Grand Conco...,None,17871
197347,128,1.577854e+12,1.550640e+12,,None,,,800 Grand Concourse Apartment Sensors Apt 5M SO,F,None,2,74.86,Zone Temperature,77,1,adc15d29e48adb0839f00644edf477ae,nyserda+0000093623+Site-Sensor+800 Grand Conco...,None,17871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197401,128,1.577855e+12,1.535774e+12,,None,,,800 Grand Concourse Boiler Steam Pressure,psi,None,5,2.09,Steam Pressure,175,3,707281647d165771ae55dd2e022ba068,nyserda+0000093623+hotWaterPlant+800 Grand Con...,None,17874
197396,128,1.577854e+12,1.535774e+12,,None,,,800 Grand Concourse Boiler Condensate Return T...,F,None,2,193.97,Condenser Water Return Temperature,388,1,1593d5b5bb0990a0b3f3d8c1bdfb0cd4,nyserda+0000093623+hotWaterPlant+800 Grand Con...,None,17874
197408,128,1.577855e+12,1.564605e+12,,None,,,800 Grand Concourse Boiler Boiler Status 2,On/Off,None,20,true,Status,121,11,8348ca9ee12b7235ed9cdc848cfd5626,nyserda+0000093623+boiler2+800 Grand Concourse...,None,17875


# Building 134

In [47]:
if use_local:
    with open('../api/buildings/134.json', 'r') as f:
        j = json.load(f)
        b_134 = pd.json_normalize(j)
else:
    b_134 = client.get_building_equipment(134)
b_134

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,17929,134,ac-2,ac-2,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,A/C-2 Active Power(kW),[],[],"[{'id': 197575, 'building_id': 134, 'last_upda...","[ahu, hvac]"
1,17930,134,ac-blower,ac-blower,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,A/C Blower Active Power(kW),[],[],"[{'id': 197576, 'building_id': 134, 'last_upda...","[ahu, hvac]"
2,17931,134,ac-condenser,ac-condenser,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,A/C Condenser Active Power(kW),[],[],"[{'id': 197578, 'building_id': 134, 'last_upda...","[ahu, hvac]"
3,17932,134,ahu-left,ahu-left,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,Air Handling Unit (Left) Active Power(kW),[],[],"[{'id': 197581, 'building_id': 134, 'last_upda...","[ahu, hvac]"
4,17933,134,ahu-right,ahu-right,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,Air Handling Unit (Right) Active Power(kW),[],[],"[{'id': 197582, 'building_id': 134, 'last_upda...","[ahu, hvac]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,18053,134,meter-tk3,meter-tk3,Meter,31,METER,meter,None,NaN,None,5,None,None,,TK-3 Active Power(kW),[],[],"[{'id': 197829, 'building_id': 134, 'last_upda...",[meter]
125,18054,134,meter-tk4,meter-tk4,Meter,31,METER,meter,None,NaN,None,5,None,None,,TK-4 Active Power(kW),[],[],"[{'id': 197831, 'building_id': 134, 'last_upda...",[meter]
126,18055,134,meter-wallOutlets,meter-wallOutlets,Meter,31,METER,meter,None,NaN,None,5,None,None,,Wall Outlets Active Power(kW),[],[],"[{'id': 197833, 'building_id': 134, 'last_upda...",[meter]
127,18056,134,uh-1,uh-1,Fan Coil Unit,9,FCU,fcu,None,NaN,None,3,None,None,,Heater 1 Active Power(kW),[],[],"[{'id': 197835, 'building_id': 134, 'last_upda...","[fcu, hvac]"


In [48]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_134_points = []
        for line in j:
            if line['building_id'] == 134:
                b_134_points.append(line)

df_b_134_points = pd.DataFrame(b_134_points).set_index('id')
df_b_134_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
197575,134,1.571198e+12,1.542394e+12,,None,,,A/C-2 Total Cost ($),,None,23,0.11,Virtual,434,18,ad3225b5482a1b2c787130fa2ff34bf2,nyserda+0000112620+ac-2+A/C-2 Total Cost ($),None,17929
197574,134,1.571198e+12,1.542394e+12,,None,,,A/C-2 Active Power(kW),kW,None,17,2.55,Real Power,221,13,bb09afea067097200622206f01ae3da7,nyserda+0000112620+ac-2+A/C-2 Active Power(kW),None,17929
197576,134,1.571198e+12,1.542650e+12,,None,,,A/C Blower Active Power(kW),kW,None,17,2.34,Real Power,221,13,a5d3e21c966c0190da259176ecf1e1d7,nyserda+0000112620+ac-blower+A/C Blower Active...,None,17930
197577,134,1.571198e+12,1.542650e+12,,None,,,A/C Blower Total Cost ($),,None,23,0.10,Virtual,434,18,407927d1e7f332c0d320ac828417b143,nyserda+0000112620+ac-blower+A/C Blower Total ...,None,17930
197578,134,1.571198e+12,1.542650e+12,,None,,,A/C Condenser Active Power(kW),kW,None,17,0.02,Real Power,221,13,53938ded3dcbd142104b30c186f4f574,nyserda+0000112620+ac-condenser+A/C Condenser ...,None,17931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197832,134,1.571198e+12,1.542394e+12,,None,,,Wall Outlets Active Power(kW),kW,None,17,0.00,Real Power,221,13,5b8e1fbf3e220a4ac729a8be92aa7da9,nyserda+0000112620+meter-wallOutlets+Wall Outl...,None,18055
197835,134,1.571198e+12,1.543287e+12,,None,,,Heater 1 Total Cost ($),,None,23,0.01,Virtual,434,18,3bee4e70e9f79337718d61c4d51130e4,nyserda+0000112620+uh-1+Heater 1 Total Cost ($),None,18056
197834,134,1.571198e+12,1.543287e+12,,None,,,Heater 1 Active Power(kW),kW,None,17,0.25,Real Power,221,13,7cd75f014a597339a726bfaee48739d5,nyserda+0000112620+uh-1+Heater 1 Active Power(kW),None,18056


# Building 445: 450 W 147th

In [49]:
if use_local:
    with open('../api/buildings/445.json', 'r') as f:
        j = json.load(f)
        b_445 = pd.json_normalize(j)
else:
    b_445 = client.get_building_equipment(445)
b_445

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,29834,445,boiler-1,1,Boiler,19,BLR,boiler,None,None,None,1,None,None,None,None,[],[],"[{'id': 326022, 'building_id': 445, 'last_upda...","[boiler, hvac]"
1,29835,445,site-Sensor,Sensor,Site Sensors,37,SITE,site,None,None,None,5,None,None,None,None,[],[],"[{'id': 326034, 'building_id': 445, 'last_upda...",[site]


In [50]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_445_points = []
        for line in j:
            if line['building_id'] == 445:
                b_445_points.append(line)

df_b_445_points = pd.DataFrame(b_445_points).set_index('id')
df_b_445_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
326022,445,1.598936e+12,1.583266e+12,None,None,None,450 W 147th Street Boiler System Thermal Lockout,None,?,?,23,0.0,Virtual,434,18,b109bf507d2e8eeacfe020fe60f906cf,nyserda+0000145732+boiler-1+450 W 147th Street...,None,29834
326021,445,1.598933e+12,1.583266e+12,None,None,None,450 W 147th Street Boiler Steam Pressure,None,psi,psi,5,0.0,Steam Pressure,175,3,e3813428ce6c3b8687b84f1f712c5b82,nyserda+0000145732+boiler-1+450 W 147th Street...,None,29834
326020,445,1.598933e+12,1.583266e+12,None,None,None,450 W 147th Street Boiler Stack Temperature,None,?,?,23,161.62,Virtual,434,18,e958c2b9a52437d8f8332267277ae17d,nyserda+0000145732+boiler-1+450 W 147th Street...,None,29834
326019,445,1.598936e+12,1.583266e+12,None,None,None,450 W 147th Street Boiler Season,None,?,?,23,1.0,Virtual,434,18,f2f7a22a37ab781c14823cf5770e748c,nyserda+0000145732+boiler-1+450 W 147th Street...,None,29834
326018,445,1.598933e+12,1.583266e+12,None,None,None,450 W 147th Street Boiler Outside Air Temperature,None,F,f,2,75.56,Outside Air Temperature,76,1,bc286e017297e8b221642f2569c96050,nyserda+0000145732+boiler-1+450 W 147th Street...,None,29834
326017,445,1.595600e+12,1.583266e+12,None,None,None,450 W 147th Street Boiler Outside Air Enable S...,None,Ordinal,ordinal,31,0.0,Mode Status,325,26,b2ffae813497a4d05122509761c37e58,nyserda+0000145732+boiler-1+450 W 147th Street...,{'1': 'true'},29834
326016,445,1.598936e+12,1.583266e+12,None,None,None,450 W 147th Street Boiler Oil,None,?,?,23,0.0,Virtual,434,18,2b2faeab2de0708444540027f4cf476b,nyserda+0000145732+boiler-1+450 W 147th Street...,None,29834
326015,445,1.598933e+12,1.583266e+12,None,None,None,450 W 147th Street Boiler Makeup Water,None,?,?,23,0.0,Virtual,434,18,ead13ae34fded23da99d0b6e0eb0844e,nyserda+0000145732+boiler-1+450 W 147th Street...,None,29834
326014,445,1.598759e+12,1.583266e+12,None,None,None,450 W 147th Street Boiler Indoor Air Enable St...,None,Ordinal,ordinal,31,0.0,Mode Status,325,26,b68a5d881c7498323b4467114f117480,nyserda+0000145732+boiler-1+450 W 147th Street...,{'1': 'false'},29834


# Building 484: Broadway?

In [51]:
if use_local:
    with open('../api/buildings/484.json', 'r') as f:
        j = json.load(f)
        b_484 = pd.json_normalize(j)
else:
    b_484 = client.get_building_equipment(484)
b_484

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,31987,484,chiller-1,1,Chiller,21,CH,chiller,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 359447, 'building_id': 484, 'last_upda...","[chiller, hvac]"
1,31988,484,chiller-2,2,Chiller,21,CH,chiller,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 359451, 'building_id': 484, 'last_upda...","[chiller, hvac]"
2,31989,484,chiller-3,3,Chiller,21,CH,chiller,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 359455, 'building_id': 484, 'last_upda...","[chiller, hvac]"
3,31990,484,chiller-4,4,Chiller,21,CH,chiller,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 359457, 'building_id': 484, 'last_upda...","[chiller, hvac]"
4,31991,484,elecMeter-8572202,8572202,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 359458, 'building_id': 484, 'last_upda...","[meter, elecMeter]"
5,31992,484,elecMeter-8572203,8572203,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 359459, 'building_id': 484, 'last_upda...","[meter, elecMeter]"
6,31993,484,elecMeter-8572204,8572204,Meter,31,METER,meter,Electrical Meter,16.0,elecMeter,5,None,None,None,None,[],[],"[{'id': 359460, 'building_id': 484, 'last_upda...","[meter, elecMeter]"
7,31994,484,site-Sensor,Sensor,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,None,None,[],[],"[{'id': 359518, 'building_id': 484, 'last_upda...",[site]
8,32159,484,supplyFan-LH-Interior,LH-Interior,Fan,26,FAN,fan,Supply Fan,15.0,supplyFan,4,None,None,None,None,[],[],"[{'id': 359847, 'building_id': 484, 'last_upda...","[fan, hvac, supplyFan]"
9,32160,484,supplyFan-LH-Perimeter,LH-Perimeter,Fan,26,FAN,fan,Supply Fan,15.0,supplyFan,4,None,None,None,None,[],[],"[{'id': 359849, 'building_id': 484, 'last_upda...","[fan, hvac, supplyFan]"


In [52]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_484_points = []
        for line in j:
            if line['building_id'] == 484:
                b_484_points.append(line)

df_b_484_points = pd.DataFrame(b_484_points).set_index('id')
df_b_484_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
359447,484,1.622620e+12,1.590998e+12,None,None,None,broadwayChilledWaterSupplyMeasurement_27th Flo...,None,None,f,2,40.0,Chilled Water Supply Temperature,89,1,15e0768bd2fbb830cc6d0019e624b0e0,nyserda+0000113572+chiller-1+broadwayChilledWa...,None,31987
359446,484,1.622620e+12,1.590998e+12,None,None,None,broadwayChilledWaterSupplyMeasurement_12th Flo...,None,None,f,2,43.0,Chilled Water Supply Temperature,89,1,47fe91bc34c082e620820289e9b64ede,nyserda+0000113572+chiller-1+broadwayChilledWa...,None,31987
359445,484,1.622620e+12,1.590998e+12,None,None,None,broadwayChilledWaterReturnMeasurement_27th Flo...,None,None,f,2,44.0,Chilled Water Return Temperature,93,1,5876b24a0e777cac9950c8cc2851c4cc,nyserda+0000113572+chiller-1+broadwayChilledWa...,None,31987
359444,484,1.622620e+12,1.590998e+12,None,None,None,broadwayChilledWaterReturnMeasurement_12th Flo...,None,None,f,2,44.0,Chilled Water Return Temperature,93,1,ada919978ef0560c6e80d6e0435b4c09,nyserda+0000113572+chiller-1+broadwayChilledWa...,None,31987
359451,484,1.622620e+12,1.590998e+12,None,None,None,broadwayChilledWaterSupplyMeasurement_27th Flo...,None,None,f,2,44.0,Chilled Water Supply Temperature,89,1,86342691d2411b119e3652edf4682b67,nyserda+0000113572+chiller-2+broadwayChilledWa...,None,31988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359848,484,1.622620e+12,1.590998e+12,None,None,None,broadwaySupplyAirFanSpeedMeasurement_Lower Hou...,None,None,%,3,20.0,Speed Sensor,636,17,88fee0908fb3fe2ae3f6409a9c749ee5,nyserda+0000113572+supplyFan-LH-Perimeter+broa...,None,32160
359851,484,1.622620e+12,1.590998e+12,None,None,None,broadwaySupplyFanStatusMeasurement_Upper House...,None,None,on/off,20,0.0,Status,121,11,52b4afdc206c5597e02e75514c40f5be,nyserda+0000113572+supplyFan-UH-Interior+broad...,None,32161
359850,484,1.622620e+12,1.590998e+12,None,None,None,broadwaySupplyAirFanSpeedMeasurement_Upper Hou...,None,None,%,3,55.08,Speed Sensor,636,17,f0835d8a8eddc29d3dfa451b2c0f7689,nyserda+0000113572+supplyFan-UH-Interior+broad...,None,32161


# Building 434: 333 Lenox Road
- Multiresidential
- Cellar + Penthouse Electric Meters

# Building 271: College/University

In [53]:
if use_local:
    with open('../api/buildings/271.json', 'r') as f:
        j = json.load(f)
        b_271 = pd.json_normalize(j)
else:
    b_271 = client.get_building_equipment(271)
b_271

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,20106,271,condenserWaterPump1,condenserWaterPump1,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,,,[],[],"[{'id': 210884, 'building_id': 271, 'last_upda...","[pump, hvac, condenserWaterPump]"
1,20107,271,condenserWaterPump2,condenserWaterPump2,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,,,[],[],"[{'id': 210889, 'building_id': 271, 'last_upda...","[pump, hvac, condenserWaterPump]"
2,20108,271,condenserWaterSystem,condenserWaterSystem,Condenser,22,COND,condenser,None,NaN,None,1,None,None,,,[],[],"[{'id': 210897, 'building_id': 271, 'last_upda...","[hvac, condenser]"
3,20109,271,csc3,csc3,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,3rd floor,[],[],"[{'id': 210918, 'building_id': 271, 'last_upda...","[ahu, hvac]"
4,20110,271,csc4,csc4,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,4th floor,[],[],"[{'id': 210941, 'building_id': 271, 'last_upda...","[ahu, hvac]"
5,20111,271,csc5,csc5,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,5th floor,[],[],"[{'id': 210965, 'building_id': 271, 'last_upda...","[ahu, hvac]"
6,20112,271,heatExchanger,heatExchanger,Heat Exchanger,28,HX,heatExchanger,None,NaN,None,1,None,None,,,[],[],"[{'id': 210972, 'building_id': 271, 'last_upda...","[heatExchanger, hvac]"
7,20113,271,hotWaterPlant,hotWaterPlant,Hot Water Plant,30,HWS,hotWaterPlant,None,NaN,None,1,None,None,,,[],"[20115, 20114]","[{'id': 210985, 'building_id': 271, 'last_upda...","[hvac, hotWaterPlant]"
8,20114,271,hotWaterPump1,hotWaterPump1,Pump,32,PUMP,pump,Hot Water Pump,25.0,hotWaterPump,1,None,None,,,[20113],[],"[{'id': 210989, 'building_id': 271, 'last_upda...","[pump, hvac, hotWaterPump]"
9,20115,271,hotWaterPump2,hotWaterPump2,Pump,32,PUMP,pump,Hot Water Pump,25.0,hotWaterPump,1,None,None,,,[20113],[],"[{'id': 210991, 'building_id': 271, 'last_upda...","[pump, hvac, hotWaterPump]"


In [54]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_271_points = []
        for line in j:
            if line['building_id'] == 271:
                b_271_points.append(line)

df_b_271_points = pd.DataFrame(b_271_points).set_index('id')
df_b_271_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
210884,271,1.551416e+12,1.527826e+12,,None,,,Condensor Water Pump 1_Failure Status,,None,23,No,Virtual,434,18,0f5871908adb713c8ab7c6fee4a70aa9,nyserda+0000080324+condenserWaterPump1+Condens...,None,20106
210887,271,1.551416e+12,1.527826e+12,,None,,,Condensor Water Pump 1_Start/Stop Output,On/Off,None,20,On,Status,121,11,013055291a69b7fe544905e6886ef903,nyserda+0000080324+condenserWaterPump1+Condens...,None,20106
210886,271,1.551416e+12,1.527826e+12,,None,,,Condensor Water Pump 1_Speed Type,,None,23,Constant,Virtual,434,18,96d61d08295c197368112c85ec745e10,nyserda+0000080324+condenserWaterPump1+Condens...,None,20106
210885,271,1.551416e+12,1.527826e+12,,None,,,Condensor Water Pump 1_Pump Status,On/Off,None,20,On,Status,121,11,d44341ae1255b37c90bf7d5959b0e5f0,nyserda+0000080324+condenserWaterPump1+Condens...,None,20106
210889,271,1.551416e+12,1.527826e+12,,None,,,Condensor Water Pump 2_Pump Status,On/Off,None,20,Off,Status,121,11,9ccc5c03098e7da0ff84b95c78a7b371,nyserda+0000080324+condenserWaterPump2+Condens...,None,20107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211050,271,1.550897e+12,1.527826e+12,,None,,,RTU-3 SIX FLOOR._Mechanical Cooling (On/Off),On/Off,None,20,Off,Status,121,11,6aed00727b6785cad3974e2ca8342104,nyserda+0000080324+rtu3+RTU-3 SIX FLOOR._Mecha...,None,20118
211053,271,1.550897e+12,1.527826e+12,,None,,,RTU-3 SIX FLOOR._Outdoor Air Enthalpy (Referen...,BTU/lb,None,28,9.3,Outside Air Enthalpy,242,24,2a7cc5659a67b7d31289e044d2d6cc9f,nyserda+0000080324+rtu3+RTU-3 SIX FLOOR._Outdo...,None,20118
211043,271,1.550897e+12,1.527826e+12,,None,,,RTU-3 SIX FLOOR._Constant Volume Operation,,None,23,Yes,Virtual,434,18,5084b0d3989834fd67e4c94de1ab1ae5,nyserda+0000080324+rtu3+RTU-3 SIX FLOOR._Const...,None,20118


# Building 114: College/University


In [57]:
if use_local:
    with open('../api/buildings/114.json', 'r') as f:
        j = json.load(f)
        b_114 = pd.json_normalize(j)
else:
    b_114 = client.get_building_equipment(114)
b_114

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,34992,114,elecMeter-1,1,Meter,31,METER,meter,Electrical Meter,16,elecMeter,5,None,None,None,None,[],[],"[{'id': 414658, 'building_id': 114, 'last_upda...","[meter, elecMeter]"


In [59]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_114_points = []
        for line in j:
            if line['building_id'] == 114:
                b_114_points.append(line)

df_b_114_points = pd.DataFrame(b_114_points).set_index('id')
df_b_114_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
414658,114,1.601507e+12,1.601507e+12,None,None,None,Main Service Power+ - power,None,None,kw,17,76.66,Real Power,221,13,be4ecdd3b1d691e9fc7d15dde0cb6b43,nyserda+0000117460+elecMeter-1+Main Service Po...,None,34992
194217,114,1.601507e+12,1.575594e+12,,None,,Mech Rm Power - power,Mech Rm Power - power,kW,kw,17,11.30,Real Power,221,13,a22c3956eefcda27e6eea810e95a683c,nyserda+0000117460+Site-Sensor+Mech Rm Power -...,None,34992


# Building 419: 221 E 71st St College/University

In [60]:
if use_local:
    with open('../api/buildings/419.json', 'r') as f:
        j = json.load(f)
        b_419 = pd.json_normalize(j)
else:
    b_419 = client.get_building_equipment(419)
b_419

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,25961,419,Site-Sensor,Site-Sensor,Site Sensors,37,SITE,site,None,NaN,None,5,None,None,,,[],[],"[{'id': 267499, 'building_id': 419, 'last_upda...",[site]
1,25962,419,ahu-1,ahu-1,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,,[],[],"[{'id': 267514, 'building_id': 419, 'last_upda...","[ahu, hvac]"
2,25963,419,ahu-10,ahu-10,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,,[],[],"[{'id': 267530, 'building_id': 419, 'last_upda...","[ahu, hvac]"
3,25964,419,ahu-2,ahu-2,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,,[],[],"[{'id': 267553, 'building_id': 419, 'last_upda...","[ahu, hvac]"
4,25965,419,ahu-3,ahu-3,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,,[],[],"[{'id': 267570, 'building_id': 419, 'last_upda...","[ahu, hvac]"
5,25966,419,ahu-4,ahu-4,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,,[],[],"[{'id': 267587, 'building_id': 419, 'last_upda...","[ahu, hvac]"
6,25967,419,ahu-5,ahu-5,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,,[],[],"[{'id': 267601, 'building_id': 419, 'last_upda...","[ahu, hvac]"
7,25968,419,ahu-6,ahu-6,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,,[],[],"[{'id': 267618, 'building_id': 419, 'last_upda...","[ahu, hvac]"
8,25969,419,ahu-7,ahu-7,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,,[],[],"[{'id': 267630, 'building_id': 419, 'last_upda...","[ahu, hvac]"
9,25970,419,ahu-8,ahu-8,Air Handling Unit,12,AHU,ahu,None,NaN,None,2,None,None,,,[],[],"[{'id': 267642, 'building_id': 419, 'last_upda...","[ahu, hvac]"


In [61]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        b_419_points = []
        for line in j:
            if line['building_id'] == 419:
                b_419_points.append(line)

df_b_419_points = pd.DataFrame(b_419_points).set_index('id')
df_b_419_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
id,,,,,,,,,,,,,,,,,,,
267499,419,1.567916e+12,1.551967e+12,,None,,,221 E 71st St Lobby Library Space Temp 3rdFl L...,F,None,2,68.78,Zone Temperature,77,1,b8ebe2c63cd21dcc3d88932ca29765dd,nyserda+0000116399+Site-Sensor+221 E 71st St L...,None,25961
267498,419,1.567916e+12,1.551967e+12,,None,,,221 E 71st St Lobby Library Space Temp 3rdFl H...,F,None,2,58.18,Zone Temperature,77,1,56b9833b5641f98568a4e2b75ebb9621,nyserda+0000116399+Site-Sensor+221 E 71st St L...,None,25961
267497,419,1.567916e+12,1.551967e+12,,None,,,221 E 71st St Lobby Library Space Temp 2ndFlMe...,F,None,2,66.20,Zone Temperature,77,1,b3ead8993beae5da955182c0a0c1fbe5,nyserda+0000116399+Site-Sensor+221 E 71st St L...,None,25961
267496,419,1.567916e+12,1.551967e+12,,None,,,221 E 71st St Lobby Library Space Temp 2ndFl S...,F,None,2,68.09,Zone Temperature,77,1,37968fc0f6475058370aee718ed32398,nyserda+0000116399+Site-Sensor+221 E 71st St L...,None,25961
267514,419,1.567994e+12,1.551970e+12,,None,,,221 E 71st St Mechanical Room Marymount AHU-1...,On/Off,None,20,false,Command,166,11,3e984c9076431ca8cf8589359146758a,nyserda+0000116399+ahu-1+221 E 71st St Mechani...,None,25962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267842,419,1.567916e+12,1.551967e+12,,None,,,221 E 71st St Cooling Tower VAV 8-8 Dmpr Signal,%,None,3,100,Damper Position,147,7,5982f0f5507cd0dac1478a3efcccfa91,nyserda+0000116399+vav-8+221 E 71st St Cooling...,None,25991
267849,419,1.567916e+12,1.551967e+12,,None,,,221 E 71st St Cooling Tower VAV8-9 Zone TempStp,F,None,2,67,Zone Temperature Setpoint,122,1,9a0943aa13855983e0719a0438f2f957,nyserda+0000116399+vav-9+221 E 71st St Cooling...,None,25992
267848,419,1.567916e+12,1.551967e+12,,None,,,221 E 71st St Cooling Tower VAV 8-9 Zone Temp,F,None,2,66.65,Zone Temperature,77,1,8e896ce10abd2ae7a34d985cbefd9351,nyserda+0000116399+vav-9+221 E 71st St Cooling...,None,25992


# Building 327: Commercial Retail
- Front/Back Supply Temp
- Front/Back Target+Current Temp
- Main Power
- Manual & SiteSage Controlled Lights